## This is my Toronto Peer Review Assginment

### Setting up my imports and let's pull the webpage so that we can scrape it  


In [1]:

import bs4 as bs
import urllib.request
import pandas as pd

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()



Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Let's use BeautifulSoup as suggested in the instructions for this assignment

In [3]:
soup = bs.BeautifulSoup(source,'lxml')

### Parse the data to extract the table data

In [4]:
#import re

table2 = soup.find('table',{"class":"wikitable sortable"})

table_rows2 = table2.find_all('tr')

l = []
for tr in table_rows2:
    td = tr.find_all('td')
    row2 = [tr.text for tr in td]
   
    l.append(row2)

del(l[0]) 


### Let's put in a df and remove the new line from the Neighbourhood entries

In [5]:
df2 = pd.DataFrame(l, columns=["Postal Code","Borough","Neighbourhood"])
df2['Neighbourhood'] = df2.Neighbourhood.str.replace(r'\n','')



### Let's find where we have __"Not assigned"__ in Borough, but with __"Not assigned"__ in Neighborhood

In [6]:
inLoc = int(df2.query("Neighbourhood == 'Not assigned' and Borough != 'Not assigned'").index[0])


### Adding Borough entry to Neighborhood per instructions when Borough has an entry and Neighborhood as Not assigned

In [7]:

df2.loc[inLoc][2] = df2.loc[inLoc][1]



### Didn't want to use a for loop since I was able to find one entry that meets the condition, but just in case, let's run it again

In [8]:
df2.query("Neighbourhood == 'Not assigned' and Borough != 'Not assigned'")

# no other entry with the above condition, GREAT! Let's continue...

,Postal Code,Borough,Neighbourhood


### query returned zero. Let us move on... removing the remaining rows with entries where we found __"Not assigned"__

In [9]:
to_drop = {'Not assigned'}
df2 = df2.loc[~df2['Borough'].isin(to_drop) & ~df2['Neighbourhood'].isin(to_drop)] 


### Resetting index number

In [10]:
df2 = df2.reset_index(drop=True)


### Combining with groupby to Neighbourhood with Borough

In [11]:
df2 = df2.groupby("Postal Code").agg(lambda x: ', '.join(set(x))).reset_index()

### Per instructions, here is the .shape to my dataframe

In [12]:
df2.shape

(103, 3)

### Importing the Geospatial_data.csv file containing the latitude and longitude for this assignment's dataframe

In [13]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2018-10-15 20:02:31--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2018-10-15 20:02:31--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-10-15 20:02:32--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.24.197, 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83p

## Load Data From CSV File

In [14]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')


### Merging the data

In [15]:
new_df = pd.merge(df2, df_geo[['Postal Code','Latitude','Longitude']],on='Postal Code')


#### How many boroughs and neighbourhoods do we have in Toronto

In [16]:
print('OBSERVATION MADE: The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(new_df['Borough'].unique()),
        new_df.shape[0]
    )
)

OBSERVATION MADE: The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [17]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/teckeon/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto